In [1]:
using Revise

In [2]:
using Images, ImageIO
using ONNXNaiveNASflux, NaiveNASflux, .NaiveNASlib
using LinearAlgebra
using OpenCV
using Flux
using CSV
using DataFrames
using DataStructures

In [16]:

function square_img(image, bbox, img_size)
    bbox_width = round(Int, bbox["bbox_x2"] - bbox["bbox_x1"])
    bbox_height = round(Int, bbox["bbox_y2"] - bbox["bbox_y1"])
    offset_tl = Int.([bbox["bbox_x1"], bbox["bbox_y1"]] .+ 1)
    h, _ = img_size
    if bbox_width >= bbox_height
        sq_img = zeros(UInt8, bbox_width, bbox_width, 3)
        if bbox_width >= h
            crop_img = image[:, offset_tl[1]: offset_tl[1] + bbox_width - 1, :]
            offset_tl[2] = 1
            bbox_height = h
        elseif offset_tl[2] + bbox_width - 1 >= h
            offset_tl[2] = h + 1 - bbox_width
            bbox_height = bbox_width
            crop_img = image[offset_tl[2] : offset_tl[2] + bbox_width - 1, offset_tl[1] : offset_tl[1] + bbox_width - 1, :]
        else
            crop_img = image[offset_tl[2] : offset_tl[2] + bbox_width - 1, offset_tl[1]+1 : offset_tl[1] + bbox_width - 1, :]
            bbox_height = bbox_width
        end
        sq_img[1:bbox_height, 1:bbox_width, :] = crop_img
    else
        sq_img = zeros(UInt8, bbox_height, bbox_height, 3)
    end
    return sq_img, offset_tl
end

function preprocess_image(img_path, csv_path)
    image = OpenCV.imread(img_path)
    _, w, h = size(image)  # (3, 1920, 1200)
    # CWH, BGR channel for Julia OpenCV, we desire WHCN for Flux model
    image = OpenCV.cvtColor(image, OpenCV.COLOR_BGR2GRAY)
    image = OpenCV.cvtColor(image, OpenCV.COLOR_GRAY2RGB)
    image = permutedims(image, (3, 2, 1))
    
    truth_df = DataFrame(CSV.File(csv_path))
    bbox = truth_df[!, [:bbox_x1, :bbox_y1, :bbox_x2, :bbox_y2]]
    bbox = bbox[parse(Int, split(img_path[1:end-4], '_')[end]) + 1, :]
    sq_image, offset_tl = square_img(image, bbox, (h, w))
    input_size = 256
    sq_image = permutedims(sq_image, (3, 2, 1))  # no need to permute multiple times, will remove it later
    resize_img = OpenCV.resize(sq_image, OpenCV.Size{Int32}(input_size, input_size), interpolation=OpenCV.INTER_AREA)
    resize_img = resize_img / 255.0
    resize_img = permutedims(resize_img, (3, 2, 1))
    return resize_img
end

function get_parallel_chains(comp_vertices, index_more_than_one_outputs)
    function get_chain(vertex)
        m = Any[]
        curr_vertex = vertex
        while length(inputs(curr_vertex)) == 1
            # println("curr vertex ", name(curr_vertex))
            push!(m, layer(curr_vertex))
            curr_vertex = outputs(curr_vertex)[1]
        end
        return Chain(m...), curr_vertex
    end
    outs = outputs(comp_vertices[index_more_than_one_outputs])
    @assert length(outs) == 2
    chain1, vertex_more_than_one_inputs = get_chain(outs[1])
    chain2, _ = get_chain(outs[2])
    @assert occursin("Add", name(vertex_more_than_one_inputs))
    inner_iter = findfirst(v -> name(v) == name(vertex_more_than_one_inputs), comp_vertices)
    if length(chain1) == 0
        return SkipConnection(chain2, (+)), inner_iter
    elseif length(chain2) == 0
        return SkipConnection(chain1, (+)), inner_iter
    else
        return Parallel(+; α = chain1, β = chain2), inner_iter
    end
end

function build_flux_model(onnx_model_path)
    comp_graph = ONNXNaiveNASflux.load(onnx_model_path)
    # find mean value
    model_vec = Any[]
    # sub_vertices = findvertices("/Sub", comp_graph)
    # if !isempty(sub_vertices)
    #     img_mean = inputs(sub_vertices[1])[2]()
    #     println(img_mean)
    #     # println(inputs(vertices(comp_graph)[5])[1]())

    #     push!(model_vec, x -> x .- img_mean)
    # end
    

    inner_iter = 0
    for (index, vertex) in enumerate(vertices(comp_graph))
        if index < 5 || index <= inner_iter
            continue
        end 
        if string(layer(vertex)) == "#213"
            push!(model_vec, NNlib.relu)
        else
            push!(model_vec, layer(vertex))
        end
        if length(outputs(vertex)) > 1
            # println("name: ", name(vertex))
            parallel_chain, inner_iter = get_parallel_chains(vertices(comp_graph), index)
            push!(model_vec, parallel_chain)
        end
    end
    model = Chain(model_vec...)
    Flux.testmode!(model)
    return (model)
end

build_flux_model (generic function with 1 method)

In [ ]:
for (index, vertex) in enumerate(vertices(comp_graph))
    println(inputs(vertex), "  ", outputs(vertex))
end

In [ ]:
for (index, vertex) in enumerate(vertices(comp_graph))
    println(outputs(vertex))
end

In [4]:
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
comp_graph = ONNXNaiveNASflux.load(onnx_model_path)

Set(Any["BatchNormalization", "Div", "Relu", "Constant", "Sub", "Conv", "ConvTranspose", "AveragePool", "Add"])


CompGraph{Vector{NaiveNASlib.AbstractVertex}, Vector{NaiveNASlib.AbstractVertex}}([NaiveNASflux.InputShapeVertex{NaiveNASlib.InputSizeVertex{NaiveNASlib.OutputsVertex{NaiveNASlib.InputVertex{String}}}, NaiveNASflux.GenericFluxConvolutional{2}}], [/final_layer/Conv])

In [5]:
function get_parallel_chains(comp_vertices, index_more_than_one_outputs)
    function get_chain(vertex)
        m = Any[]
        curr_vertex = vertex
        while length(inputs(curr_vertex)) == 1
            # println("curr vertex ", name(curr_vertex))
            push!(m, layer(curr_vertex))
            curr_vertex = outputs(curr_vertex)[1]
        end
        return Chain(m...), curr_vertex
    end
    outs = outputs(comp_vertices[index_more_than_one_outputs])
    @assert length(outs) == 2
    chain1, vertex_more_than_one_inputs = get_chain(outs[1])
    chain2, _ = get_chain(outs[2])
    @assert occursin("Add", name(vertex_more_than_one_inputs))
    inner_iter = findfirst(v -> name(v) == name(vertex_more_than_one_inputs), comp_vertices)
    if length(chain1) == 0
        return SkipConnection(chain2, (+)), inner_iter
    elseif length(chain2) == 0
        return SkipConnection(chain1, (+)), inner_iter
    else
        return Parallel(+; α = chain1, β = chain2), inner_iter
    end
end

function build_flux_model(onnx_model_path)
    comp_graph = ONNXNaiveNASflux.load(onnx_model_path)

    # find mean value
    model_vec = Any[]
    # sub_vertices = findvertices("/Sub", comp_graph)
    # if !isempty(sub_vertices)
    #     img_mean = inputs(sub_vertices[1])[2]()
    #     println(img_mean)
    #     # println(inputs(vertices(comp_graph)[5])[1]())

    #     push!(model_vec, x -> x .- img_mean)
    # end
    img_mean = reshape([0.48500, 0.45600, 0.40600], (1, 1, 3))
    push!(model_vec, x -> x .- img_mean)

    img_variance = reshape([0.2990, 0.22400, 0.22500], (1, 1, 3))
    push!(model_vec, x -> x ./ img_variance)

    inner_iter = 0
    for (index, vertex) in enumerate(vertices(comp_graph))
        if index < 5 || index <= inner_iter
            continue
        end 
        # println(index, "   ",layer(vertex))
        push!(model_vec, layer(vertex))
        if length(outputs(vertex)) > 1
            # println(name(vertex))
            parallel_chain, inner_iter = get_parallel_chains(vertices(comp_graph), index)
            push!(model_vec, parallel_chain)
        end
    end
    model = Chain(model_vec...)
    Flux.testmode!(model)
    return (model)
end

build_flux_model (generic function with 1 method)

In [7]:
# +++++++++++++++++++ build flux model +++++++++++++++++++
onnx_model_path = "/home/verification/ModelVerification.jl/onnx_parser/resnet_model.onnx"
model = build_flux_model(onnx_model_path)
println.(model)
#= queue = Queue{Any}()
comp_graph = ONNXNaiveNASflux.load(onnx_model_path)
for (index, vertex) in enumerate(vertices(comp_graph))
    println(inputs(vertex), "  ", outputs(vertex))
    println(layer(vertex))
    enqueue!(queue, layer(vertex))
end
println(dequeue!(queue)) =#

Set(Any["BatchNormalization", "Div", "Relu", "Constant", "Sub", "Conv", "ConvTranspose", "AveragePool", "Add"])
#19


#20
Conv(

(7, 7), 3 => 64, pad=3, stride=2, bias=false)
BatchNorm(64

, relu, active=false)
MeanPool((3, 3), pad=1, stride=2)


SkipConnection(Chain(Conv((3, 3), 64 => 64, pad=1, bias=false), BatchNorm(64, relu, active=false), Conv((3, 3), 64 => 64, pad=1, bias=false), BatchNorm(64, active=false)), +)
#213
SkipConnection(Chain(Conv((3, 3), 64 => 64, pad=1, bias=false), BatchNorm(64, relu, active=false), Conv((3, 3), 64 => 64, pad=1, bias=false), BatchNorm(64, active=false)), +)
#213


Parallel(+, α = Chain(Conv((3, 3), 64 => 128, pad=1, stride=2, bias=false), BatchNorm(128, relu, active=false), Conv((3, 3), 128 => 128, pad=1, bias=false), BatchNorm(128, active=false))

, β = Chain(Conv((1, 1), 64 => 128, stride=2, bias=false), BatchNorm(128, active=false)))
#213
SkipConnection(Chain(Conv((3, 3), 128 => 128, pad=1, bias=false), BatchNorm(128, relu, active=false), Conv((3, 3), 128 => 128, pad=1, bias=false), BatchNorm(128, active=false)), +)
#213
Parallel(+, α = Chain(Conv((3, 3), 128 => 256, pad=1, stride=2, bias=false), BatchNorm(256, relu, active=false), Conv((3, 3), 256 => 256, pad=1, bias=false), BatchNorm(256, active=false)), β = Chain(Conv((1, 1), 128 => 256, stride=2, bias=false), BatchNorm(256, active=false)))
#213
SkipConnection(Chain(Conv((3, 3), 256 => 256, pad=1, bias=false), BatchNorm(256, relu, active=false), Conv((3, 3), 256 => 256, pad=1, bias=false), BatchNorm(256, active=false)), +)
#213
Parallel(+, α = Chain(Conv((3, 3), 256 => 512, pad=1, stride=2, bias=false), BatchNorm(512, relu, active=false), Conv((3, 3), 512 => 512, pad=1, bias=false), BatchNorm(512, active=false)), β = Chain(Conv((1, 1), 256 => 512, stride=2, bias=false), Bat

(1, 1), 64 => 24)


32-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [5]:
using LazySets
using ModelVerification
using PyCall
using CSV
using ONNX
using Flux
using Test
using CUDA
using MLDatasets: CIFAR10
using MLUtils: splitobs, DataLoader
using Accessors
using Profile

In [6]:
# # +++++++++++++++++++ preprocess image +++++++++++++++++++
img_path = "./AircraftInspection_00000008.png"
csv_path = "./SynthPlane_08.csv"
img = preprocess_image(img_path, csv_path)
input = reshape(img, (size(img)...,1))   # WHCN for Julia, NCHW for Python
input = permutedims(input, (2, 1, 3, 4))
img_mean = reshape([0.48500, 0.45600, 0.40600], (1, 1, 3, 1))
img_variance = reshape([0.2990, 0.22400, 0.22500], (1, 1, 3, 1))
input = (input .- img_mean) ./ img_variance
output = model(Float32.(input))
println(output[1:5,1:5,1,1])

Float32[-0.21893519 -0.15008841 -0.36900473 -0.33628064 -0.13221027; -0.1850356 -0.101651594 -0.30785286 -0.23966312 -0.21652326; -0.1388096 -0.1630137 -0.1909296 -0.028720766 -0.012907296; -0.047417775 -0.051144868 -0.15390214 0.057320535 0.0035933554; -0.08829597 -0.05016379 -0.05527121 -0.052590728 -0.18598792]


In [15]:
input = reshape(input, 256, 256, 3)
input_perturbed = copy(input)
# input_perturbed[100:102,100:102,1:3,1] .= -0.5
# input_perturbed[1,1,1,1] = -0.5
# input_perturbed[100,100,1,1] .= -0.5
input_perturbed[100,100:101,1:3,1] .= -0.5

2×3 view(::Array{Float64, 4}, 100, 100:101, 1:3, 1) with eltype Float64:
 -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5

In [13]:
testmode!(model)
image_seeds = [input, input_perturbed]  # 256 x 256 x 3 x 2
println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)

Float64


BallInf{Float64, Vector{Float64}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0)

In [16]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

Image Zono reach
Inf


(value = BasicResult(:holds), time = 160.782878615, bytes = 139109526184, gctime = 6.457323193, gcstats = Base.GC_Diff(139109526184, 266, 0, 18641819, 76, 284, 6457323193, 4, 0))